### 1. Dependancies and Setup

In [ ]:
import os
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout
from tensorflow.keras.applications import ResNet152V2

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

### 2. Load Data

##### 2.1 Load Images

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
pattern_train = os.path.join('aug_data', 'train', 'images', '*.jpg')
train_images = tf.data.Dataset.list_files(pattern_train, shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (250,250)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
pattern_test = os.path.join('aug_data', 'test', 'images', '*.jpg')
test_images = tf.data.Dataset.list_files(pattern_test, shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (250,250)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
pattern_val = os.path.join('aug_data', 'val', 'images', '*.jpg')
val_images = tf.data.Dataset.list_files(pattern_val, shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (250,250)))
val_images = val_images.map(lambda x: x/255)

##### 2.2 Load Labels

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

In [ ]:
pattern_train = os.path.join('aug_data', 'train', 'labels', '*.json')
train_labels = tf.data.Dataset.list_files(pattern_train, shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
pattern_test = os.path.join('aug_data', 'test', 'labels', '*.json')
test_labels = tf.data.Dataset.list_files(pattern_test, shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

In [ ]:
pattern_val = os.path.join('aug_data', 'val', 'labels', '*.json')
val_labels = tf.data.Dataset.list_files(pattern_val, shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.float16]))

##### 2.3 Combine Images and Labels

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(3360)
train = train.batch(16)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1155)
test = test.batch(16)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1120)
val = val.batch(16)
val = val.prefetch(4)

##### 2.4 Samples

In [ ]:
data_samples = train.as_numpy_iterator()
res = data_samples.next()

In [ ]:
print(res[1][0][1])

In [ ]:
plt.imshow(res[0][0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx].copy()
    sample_coords = res[1][0][idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)

In [ ]:
train = train.map(lambda x, y: (x, tf.reshape(y, (-1, 4))))
test = test.map(lambda x, y: (x, tf.reshape(y, (-1, 4))))
val = val.map(lambda x, y: (x, tf.reshape(y, (-1, 4))))

### 3. Model

##### 3.1 Neural Network

In [ ]:
model = Sequential([
    Input(shape=(250,250,3)), 
    ResNet152V2(include_top=False, input_shape=(250,250,3)),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(256, 3, 2, padding='same', activation='relu'),
    Conv2D(256, 2, 2, activation='relu'),
    Dropout(0.05),
    Conv2D(4, 2, 2),
    Reshape((4,))
])

##### 3.2 Optimizer and Losses

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer, loss)

In [ ]:
X, y = train.as_numpy_iterator().next()
X.shape

In [ ]:
scores = model.predict(X)
scores

In [ ]:
hist = model.fit(train, epochs=100, validation_data=val)

In [ ]:
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val loss')
plt.suptitle('Loss')
plt.legend()
plt.show()

In [ ]:
test_data = test.as_numpy_iterator()
test_sample = test_data.next()
yhat = model.predict(test_sample[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx].copy()
    sample_coords = yhat[idx]
    
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[:2], [250,250]).astype(int)), 2, (255,0,0), -1)
    cv2.circle(sample_image, tuple(np.multiply(sample_coords[2:], [250,250]).astype(int)), 2, (0,255,0), -1)
    
    ax[idx].imshow(sample_image)